In [75]:
import io
import numpy as np
import pandas as pd
import pycurl

In [2]:
ip = 'frontend.hpc4ai.unito.it:4567'
pandora_url = '/pandora_console'
apipass = '12345678'
user = 'stefaniak'
password = 'simplepass'
op = 'get'

# Set up base URL for querying data
url = "https://" + ip  + pandora_url + "/include/api.php"

url += "?"
url += "apipass=" + apipass
url += "&user="  + user
url += "&pass=" + password
url += "&op=" + op

# Save URL with op=get for later
get_url = url 

In [9]:
# Helper functions

# Set URL with additional parameters in the dictionary
def url_set(params_dict):
    url = get_url
    for k, v in params_dict.items():
        url += '&' + k + '=' + v
    
    return url

# Get output of a given URL in text format
def url_out(url):
    buf = io.BytesIO()

    c = pycurl.Curl()
    c.setopt(c.URL, url)
    c.setopt(c.WRITEFUNCTION, buf.write)
    c.perform()

    output = buf.getvalue().decode('UTF-8')
    buf.close()
    
    return output

In [ ]:
# Query all agents
url_get_agents = url_set({"op2": "all_agents", "return_type": "csv"})
print(url_get_agents)

output = url_out(url_get_agents)

In [11]:
# Process agents into a list of dictionaries

lines = output.split("\n")
agents = []
for line in lines:
    if not line:
        continue
    
    fields = line.split(";")
    agent = {}
    agent['id_agent'] = fields[0]
    agent['name'] = fields[1]
    agent['ip'] = fields[2]
    agent['description'] = fields[3]
    agent['os_name'] = fields[4]
    agent['url_address'] = fields[5]

    agents.append(agent)

In [12]:
print(agents)

[{'id_agent': '56', 'name': 'broadwell-000', 'ip': '192.168.100.155', 'description': 'Created by pandorafms.maas', 'os_name': 'Linux', 'url_address': ''}, {'id_agent': '57', 'name': 'broadwell-001', 'ip': '192.168.100.156', 'description': 'Created by pandorafms.maas', 'os_name': 'Linux', 'url_address': ''}, {'id_agent': '58', 'name': 'broadwell-002', 'ip': '192.168.100.22', 'description': 'Created by pandorafms.maas', 'os_name': 'Linux', 'url_address': ''}, {'id_agent': '59', 'name': 'broadwell-003', 'ip': '192.168.100.23', 'description': 'Created by pandorafms.maas', 'os_name': 'Linux', 'url_address': ''}, {'id_agent': '60', 'name': 'broadwell-004', 'ip': '192.168.100.24', 'description': 'Created by pandorafms.maas', 'os_name': 'Linux', 'url_address': ''}, {'id_agent': '61', 'name': 'broadwell-005', 'ip': '192.168.100.25', 'description': 'Created by pandorafms.maas', 'os_name': 'Linux', 'url_address': ''}, {'id_agent': '62', 'name': 'broadwell-006', 'ip': '192.168.100.26', 'descriptio

In [15]:
# Test: query agent modules from a fixed agenet

id_agent = '59'
url_get_agent_modules = url_set({'op2': 'agent_modules', 'return_type': 'csv', 
                                 'other': id_agent, 
                                 'other_mode': 'url_encode_separator_|'})
output = url_out(url_get_agent_modules)

print(output)

59;711;cpu_user
59;712;Load Average
59;713;proctotal
59;714;sshDaemon
59;715;LastLogin
59;716;Cron task files
59;717;/dev/sda2
59;718;/dev/loop1
59;719;/dev/loop3
59;720;/dev/loop4
59;721;tmpfs
59;722;/dev/loop5
59;723;udev
59;724;beegfs_data01
59;725;/dev/loop2
59;1919;/dev/loop0
59;2510;/dev/loop6
59;2632;Syslog
59;2690;192.168.201.101@o2ib1:/scratch



In [32]:
# Query all agent modules

modules = []
for agent in agents:
    url_get_agent_modules = url_set({'op2': 'agent_modules', 'return_type': 'csv', 
                                     'other': agent['id_agent'], 
                                     'other_mode': 'url_encode_separator_|'})
    output = url_out(url_get_agent_modules)
    
    # Process modules into a list of dictionaries
    lines = output.split("\n")
    for line in lines:
        if not line:
            continue
    
        fields = line.split(";")
        module = {}
        module['id_agent'] = fields[0]
        module['name_agent'] = agent['name']
        module['id_module'] = fields[1]
        module['name_module'] = fields[2]

        modules.append(module)

In [35]:
# Search module with given parameters
def search_module(name_agent, name_module):
    for module in modules:
        if (name_agent == module['name_agent']) and (name_module == module['name_module']):
            return module
    raise Exception('Cannot find specified module')

In [64]:
# Query data from desired module in a given time frame

target = search_module('broadwell-000', 'cpu_user')
time_from = '20230615T12:00'
time_to   = '20230716T12:00'

url_get_module_data = url_set({'op2': 'module_data', 'id': target['id_module'], 
                               'other': f';||{time_from}|{time_to}', 
                               'other_mode': 'url_encode_separator_|'})
output = url_out(url_get_module_data)

In [65]:
# Process module data

lines = output.split("\n")

# Pre-init for efficiency
times = [None] * len(lines)
vals  = [None] * len(lines)

for i, line in enumerate(lines):
    if not line:
        continue
    
    fields = line.split(";")
    times[i] = fields[0]
    vals[i]  = fields[1]

In [91]:
# Construct a data frame
df = pd.DataFrame({'Time': times, 'Value': vals})

# Convert values to numeric format and remove invalid rows
df['Value'] = pd.to_numeric(df['Value'], errors = 'coerce')
df = df.dropna(subset = ['Value'])

# Convert times to date-time format
df['Time'] = pd.to_numeric(df['Time'])
df['Time'] = pd.to_datetime(df['Time'], unit = 's')

In [93]:
print(df)

                    Time  Value
6339 2023-07-07 10:13:09    1.0
6340 2023-07-07 10:15:00    1.0
6341 2023-07-07 10:18:10    0.0
6342 2023-07-07 10:20:00    0.0
6343 2023-07-07 10:25:00    0.0
...                  ...    ...
9162 2023-07-16 09:35:00    0.0
9163 2023-07-16 09:40:00    0.0
9164 2023-07-16 09:45:00    0.0
9165 2023-07-16 09:50:00    0.0
9166 2023-07-16 09:55:00    0.0

[2828 rows x 2 columns]
